In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_path = '../data/clean_recipes.csv'
df = pd.read_csv(df_path, sep=';')

In [4]:
print(df.shape)
df.head()

(12351, 10)


,Recipe Name,Review Count,Recipe Photo,Author,Prepare Time,Cook Time,Total Time,Ingredients,Directions,RecipeID
0,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,25 m,15 m,3 h 10 m,"yeast,water,white sugar,salt,egg,butter,flour,...","Dissolve yeast in warm water.**Stir in sugar, ...",7000
1,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,15 m,1 h,1 h 20 m,"flour,salt,baking powder,poppy,butter,vegetabl...",'Preheat oven to 350 degrees F (175 degrees C)...,7001
2,Applesauce Bread I Recipe,124,https://images.media-allrecipes.com/userphotos...,GAF55,10 m,1 h 20 m,1 h 30 m,"flour,egg,white sugar,vegetable oil,applesauce...",Preheat oven to 350 degrees F (175 degrees C)....,7003
3,Apple Raisin Bread Recipe,39,https://images.media-allrecipes.com/userphotos...,Helen Hanson,15 m,1 h,1 h 15 m,"flour,baking powder,baking soda,salt,cinnamon,...",Preheat oven to 350 degrees F (175 degrees C)....,7006
4,Buttermilk Oatmeal Bread Recipe,41,https://images.media-allrecipes.com/userphotos...,Helen Hanson,10 m,1 h,1 h 40 m,"oat,buttermilk,vegetable oil,egg,brown sugar,f...",Mix oats with buttermilk. Let stand for 1/2 h...,7007


In [34]:
ingr_df = df[['Recipe Name', 'Ingredients']]

In [52]:
ingr_df.dropna(inplace = True)

new_df = ingr_df['Ingredients'].str.split(",", n = -1, expand = True)
new_df2 = ingr_df['Ingredients'].str.split(",", n = -1, expand = False)

new_df.head(15)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,yeast,water,white sugar,salt,egg,butter,flour,butter,None,None,...,None,None,None,None,None,None,None,None,None,None
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,...,None,None,None,None,None,None,None,None,None,None
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,...,None,None,None,None,None,None,None,None,None,None
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,...,None,None,None,None,None,None,None,None,None,None
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,None,...,None,None,None,None,None,None,None,None,None,None
5,shortening,white sugar,salt,milk,egg,lemon,yeast,flour,fruit,None,...,None,None,None,None,None,None,None,None,None,None
6,shortening,water,brown sugar,yeast,water,white sugar,salt,bread,whole wheat,None,...,None,None,None,None,None,None,None,None,None,None
7,water,cottage cheese,vegetable oil,egg,flour,white sugar,baking soda,salt,yeast,None,...,None,None,None,None,None,None,None,None,None,None
8,margarine,white sugar,egg,banana,coffee,water,vanilla,flour,salt,baking powder,...,None,None,None,None,None,None,None,None,None,None
9,cornmeal,milk,egg,bell pepper,onion,garlic,salt,baking soda,white sugar,corn,...,None,None,None,None,None,None,None,None,None,None


In [61]:
new_df2.head()

0    [yeast, water, white sugar, salt, egg, butter,...
1    [flour, salt, baking powder, poppy, butter, ve...
2    [flour, egg, white sugar, vegetable oil, apple...
3    [flour, baking powder, baking soda, salt, cinn...
4    [oat, buttermilk, vegetable oil, egg, brown su...
Name: Ingredients, dtype: object

In [55]:
new_df2[0][3]

'salt'

In [60]:
new_df2.isin(['flour', 'salt', 'egg'])

SystemError: <built-in method view of numpy.ndarray object at 0x00000180DFC8A9E0> returned a result with an error set

* User starts to type in first ingredient
* we filter all rows that contain that given ingredient
* temp_df with filtered results

* user types in the second ingredient
* we filter all rows that contain that given ingredient from temp_df

* repeat

flour >> salt >> brown sugar

at this point - 